In [1]:
import pandas as pd
from scipy import stats

### load data

In [2]:
cohort_df = pd.read_csv('cohort_18_to_90_df.csv',dtype = {'Phecode':str})

/tmp/ipykernel_426/536062527.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  cohort_df = pd.read_csv('cohort_18_to_90_df.csv',dtype = {'Phecode':str})


In [25]:
cohort_df.condition_start_datetime.max()

'2022-07-01 23:59:59+00:00'

In [3]:
#less_200_df = pd.read_csv('result_v2/less_than_200.csv', dtype= {'Phecode':str})

In [3]:
# matched_pid = pd.read_csv('matched_person_ids.csv').rename(columns = {'x':'person_id'})

In [3]:
matched_pid_f = pd.read_csv('matched_person_ids_female.csv').rename(columns = {'x':'person_id'})

In [4]:
matched_pid_m = pd.read_csv('matched_person_ids_male.csv').rename(columns = {'x':'person_id'})

In [26]:
icd2phe = pd.read_csv('icd2phe.csv')

In [28]:
icd2phe.Phecode.nunique()

1875

### def for create matrix

In [7]:
# Define the mapping dictionary
sex_mapping = {
    'Male': 'Male',
    'Female': 'Female',
    'I prefer not to answer': 'Other sex',
    'Intersex': 'Other sex',
    'No matching concept': 'Other sex'
}

race_mapping = {
    'White':'White' ,                                     
    'Black or African American':'Black or African American' , 
    'Asian':'Asian' ,                                    
    'More than one population':'More than one population',                      
    'None of these':'Other race',    
    'None Indicated':'No answer race',                                                              
    'PMI: Skip': 'No answer race',
    'I prefer not to answer':'No answer race',                        
    'Middle Eastern or North African':'Other race',               
    'Native Hawaiian or Other Pacific Islander':'Other race'    
    }

ethinicity_mapping = {
    'Not Hispanic or Latino':'Not Hispanic or Latino',                            
    'Hispanic or Latino':'Hispanic or Latino',                                                                            
    'What Race Ethnicity: Race Ethnicity None Of These':'Other ethinicity', 
    'No matching concept': 'Other ethinicity',
    'PMI: Prefer Not To Answer':'No answer ethinicity', 
    'PMI: Skip':'No answer ethinicity'
    }


In [8]:
def before_create_matrix(df):
    df = df[['person_id','gender', 'date_of_birth', 'race', 'ethnicity','sex_at_birth','birth_year', 'age_last_EHR', 'HS']].drop_duplicates()
    df['sex_revised'] = df['sex_at_birth'].map(sex_mapping)
    df['race_revised'] = df['race'].map(race_mapping)
    df['ethinicity_revised'] = df['ethnicity'].map(ethinicity_mapping)
#     print(df['sex_revised'].value_counts(),'\n')
#     print(df['race_revised'].value_counts(),'\n')
#     print(df['ethinicity_revised'].value_counts(),'\n')
#     print(df.groupby(['sex_revised','HS'])['person_id'].nunique(),'\n')
#     print(df.groupby(['race_revised','HS'])['person_id'].nunique(),'\n')
#     print(df.groupby(['ethinicity_revised','HS'])['person_id'].nunique(),'\n')
    sex_matrix = df.assign(value=1).pivot_table(index='person_id', columns='sex_revised', values='value', fill_value=0)
    race_matrix = df.assign(value=1).pivot_table(index='person_id', columns='race_revised', values='value', fill_value=0)
    ethinicity_matrix = df.assign(value=1).pivot_table(index='person_id', columns='ethinicity_revised', values='value', fill_value=0)   
    is_hs = df[['person_id','HS']].drop_duplicates()
    df_sub = (is_hs
          .merge(race_matrix, on='person_id', how='left')
          .merge(sex_matrix, on='person_id', how='left')
          .merge(ethinicity_matrix, on='person_id', how='left'))
    # Apply z-score normalization
    df['age_normalized'] = stats.zscore(df['age_last_EHR'])
    df = df[['person_id','age_normalized']]
    df_merge = df_sub.merge(df, on = 'person_id', how = 'left')
    return df_merge

In [9]:
def create_matrix(df,df_demo):
    # Drop duplicate entries for person_id and Phecode
    df = df.drop_duplicates(subset=['person_id', 'Phecode'])
    
    # Create the binary matrix using pivot_table
    binary_matrix = df.assign(value=1).pivot_table(index='person_id', 
                                                   columns='Phecode', values='value', fill_value=0)
    
    binary_matrix = binary_matrix.merge(df_demo, how = 'left', on = 'person_id')
    return binary_matrix

### Create matrix for remove record less than 200

#### select dataset

In [8]:
less_200_list = less_200_df['Phecode']

In [9]:
less_than_200_cohort = cohort_df[~cohort_df['Phecode'].isin(less_200_list)]

In [10]:
len(less_than_200_cohort['person_id'].unique())

216938

In [23]:
less_than_200_demo = before_create_matrix(less_than_200_cohort)

In [24]:
less_than_200_demo.head(5)

,person_id,HS,Asian,Black or African American,More than one population,No answer race,Other race,White,Female,Male,Other sex,Hispanic or Latino,No answer ethinicity,Not Hispanic or Latino,Other ethinicity,age_normalized
0,7189522,0,0.0,0.0,0.0,0.0,0.0,1.0,1,0,0,0,0,1,0,0.642312
1,9306478,0,0.0,0.0,0.0,0.0,0.0,1.0,1,0,0,0,0,1,0,0.548530
2,9557736,0,0.0,0.0,0.0,0.0,0.0,1.0,0,1,0,0,0,1,0,-0.015306
3,1840872,0,0.0,0.0,0.0,0.0,0.0,1.0,0,1,0,0,0,1,0,0.283089
4,3486599,0,0.0,0.0,0.0,0.0,0.0,1.0,0,1,0,0,0,1,0,1.841141


In [60]:
less_than_200_phecode = create_matrix(less_than_200_cohort,less_than_200_demo)

In [61]:
less_than_200_phecode.head(5) #1864

,person_id,008,008.5,008.51,008.52,008.6,008.7,010,031,038,...,Other race,White,Female,Male,Other sex,Hispanic or Latino,No answer ethinicity,Not Hispanic or Latino,Other ethinicity,age_normalized
0,1000000,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0,1,0,0,0,1,0,0.522626
1,1000004,0,0,0,0,0,0,0,0,0,...,0.0,1.0,0,1,0,0,0,1,0,1.599145
2,1000033,0,0,0,0,0,0,0,0,1,...,0.0,1.0,0,1,0,0,0,1,0,0.642968
3,1000039,0,0,0,0,0,0,0,0,0,...,0.0,0.0,1,0,0,0,0,1,0,-0.329769
4,1000042,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0,1,0,0,0,1,0,0.044045


In [62]:
less_than_200_phecode.to_csv('result_v2/less_than_200_phecode_matrix.csv', index = False)

### Create matrix for matchit cohort

In [6]:
matchit_condition_df = cohort_df[cohort_df['person_id'].isin(matched_pid['person_id'])]

In [7]:
matchit_condition_df.head()

,person_id,standard_concept_code,source_concept_code,source_vocabulary,condition_start_datetime,ICDcode,ICDsource,Phecode,Phenotype,category,...,length_first_to_last,gender,date_of_birth,race,ethnicity,sex_at_birth,birth_year,age_2024,HS,age_last_EHR
9,2395521,118185001,V22.1,ICD9CM,2008-12-05 06:00:00+00:00,V22.1,ICD9CM,NaN,NaN,NaN,...,39.342466,Female,1973-06-15 00:00:00+00:00,White,Not Hispanic or Latino,Female,1973,51,0,49.054795
10,2563639,52254009,581.9,ICD9CM,2013-09-19 10:36:00+00:00,581.9,ICD9CM,580.2,Nephrotic syndrome without mention of glomerul...,genitourinary,...,14.353425,Female,1985-06-15 00:00:00+00:00,Black or African American,Not Hispanic or Latino,Female,1985,39,0,36.989041
17,1734015,111222003,717.3,ICD9CM,2008-04-11 08:30:00+00:00,717.3,ICD9CM,835,Internal derangement of knee,injuries & poisonings,...,22.526027,Male,1947-06-15 00:00:00+00:00,White,Not Hispanic or Latino,Male,1947,77,0,75.095890
18,2010154,57190000,367.1,ICD9CM,2014-11-03 06:00:00+00:00,367.1,ICD9CM,367.1,Myopia,sense organs,...,21.252055,Female,1968-06-15 00:00:00+00:00,White,Not Hispanic or Latino,Female,1968,56,0,54.035616
29,1067462,421895002,250.70,ICD9CM,2015-02-09 06:00:00+00:00,250.70,ICD9CM,250.25,Diabetes type 2 with peripheral circulatory di...,endocrine/metabolic,...,33.969863,Female,1957-06-15 00:00:00+00:00,White,Not Hispanic or Latino,Female,1957,67,0,65.087671


In [11]:
matchit_condition_df[~matchit_condition_df['Phecode'].isna()]['ICDcode'].unique()

array(['581.9', '717.3', '367.1', ..., 'M99.19', 'S91.106?', 'S13.100?'],
      dtype=object)

In [13]:
matchit_condition_df[matchit_condition_df['Phecode'].isna()].sort_values(by = 'ICDcode')['ICDcode'].unique()

array(['629.20', '629.22', '629.23', '629.31', '629.32', '629.81',
       '635.00', '635.01', '635.02', '635.10', '635.11', '635.12',
       '635.20', '635.22', '635.7', '635.70', '635.72', '635.80',
       '635.81', '635.82', '635.9', '635.90', '635.91', '635.92',
       '636.20', '636.40', '637.00', '637.01', '637.10', '637.11',
       '637.70', '637.71', '637.81', '637.9', '637.90', '637.91',
       '637.92', '638.9', '649.81', '765.20', '997.32', '997.41',
       '997.49', '999.32', '999.33', '999.52', '999.59', 'A92.5',
       'B37.31', 'C44', 'C44.1022', 'C44.1122', 'C44.1191', 'C44.1192',
       'C44.1221', 'C44.1222', 'C56.3', 'C96', 'D04.112', 'D22.112',
       'D22.122', 'D23.111', 'D23.112', 'D23.121', 'D23.122', 'D32',
       'D47.02', 'D47.Z2', 'D48.19', 'D57.03', 'D57.09', 'D57.218',
       'D57.439', 'D57.458', 'D57.459', 'D57.81', 'D57.818', 'D59.10',
       'D59.11', 'D59.12', 'D59.19', 'D63', 'D68.00', 'D68.6', 'D72.10',
       'D72.119', 'D72.12', 'D75', 'D75.829', '

In [14]:
matchit_condition_df = matchit_condition_df.dropna(subset = ['Phecode'])

In [26]:
matchit_demo = before_create_matrix(matchit_condition_df)

In [28]:
matchit_demo.columns

Index(['person_id', 'HS', 'Asian', 'Black or African American',
       'More than one population', 'No answer race', 'Other race', 'White',
       'Female', 'Male', 'Other sex', 'Hispanic or Latino',
       'No answer ethinicity', 'Not Hispanic or Latino', 'Other ethinicity',
       'age_normalized'],
      dtype='object')

In [29]:
matchit_phecode = create_matrix(matchit_condition_df,matchit_demo)

In [30]:
matchit_phecode.head(5)

,person_id,008,008.5,008.51,008.52,008.6,008.7,010,031,038,...,Other race,White,Female,Male,Other sex,Hispanic or Latino,No answer ethinicity,Not Hispanic or Latino,Other ethinicity,age_normalized
0,1000095,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,1,0,-0.315170
1,1000117,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0.194519
2,1000151,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0.554551
3,1000262,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,1,0,-1.427234
4,1000357,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,1,0,1.898064


In [31]:
matchit_phecode.to_csv('matchit_phecode_matrix.csv', index = False)

### Create matrix for matchit female

In [10]:
matchit_female_df = cohort_df[cohort_df['person_id'].isin(matched_pid_f['person_id'])]

In [11]:
matchit_female_df = matchit_female_df.dropna(subset = ['Phecode'])

In [12]:
matchit_female_demo = before_create_matrix(matchit_female_df)

In [13]:
matchit_female_demo.head(5)

,person_id,HS,Asian,Black or African American,More than one population,No answer race,Other race,White,Female,Hispanic or Latino,No answer ethinicity,Not Hispanic or Latino,Other ethinicity,age_normalized
0,2563639,0,0,1,0,0,0,0,1,0,0,1,0,-0.688852
1,2010154,0,0,0,0,0,0,1,1,0,0,1,0,0.461045
2,1067462,0,0,0,0,0,0,1,1,0,0,1,0,1.206575
3,5412506,0,0,0,0,0,0,1,1,0,0,1,0,0.261448
4,6315019,0,0,0,0,0,0,1,1,0,0,1,0,0.261448


In [14]:
matchit_female_phecode = create_matrix(matchit_female_df,matchit_female_demo)

In [15]:
matchit_female_phecode = matchit_female_phecode.drop(columns = 'Female')

In [23]:
matchit_female_phecode.to_csv('matchit_phecode_matrix_female.csv', index = False)

### Create matrix for matchit male

In [16]:
matchit_male_df = cohort_df[cohort_df['person_id'].isin(matched_pid_m['person_id'])]

In [17]:
matchit_male_df = matchit_male_df.dropna(subset = ['Phecode'])

In [18]:
matchit_male_demo = before_create_matrix(matchit_male_df)

In [19]:
matchit_male_demo.head(5)

,person_id,HS,Asian,Black or African American,More than one population,No answer race,Other race,White,Male,Hispanic or Latino,No answer ethinicity,Not Hispanic or Latino,Other ethinicity,age_normalized
0,1734015,0,0,0,0,0,0,1,1,0,0,1,0,1.443020
1,3002516,1,0,0,0,0,0,1,1,0,0,1,0,0.903112
2,1291509,0,0,1,0,0,0,0,1,0,0,1,0,0.766981
3,2125039,0,0,1,0,0,0,0,1,0,0,1,0,0.363573
4,9642278,0,0,0,0,0,0,1,1,0,0,1,0,-0.718829


In [20]:
matchit_male_phecode = create_matrix(matchit_male_df,matchit_male_demo)

In [21]:
matchit_male_phecode = matchit_male_phecode.drop(columns = 'Male')

In [22]:
matchit_male_phecode.head()

,person_id,008,008.5,008.51,008.52,008.6,008.7,010,031,038,...,Black or African American,More than one population,No answer race,Other race,White,Hispanic or Latino,No answer ethinicity,Not Hispanic or Latino,Other ethinicity,age_normalized
0,1000320,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,-1.177280
1,1000976,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0.417878
2,1001355,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,-0.348300
3,1001485,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0.496934
4,1001917,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,-0.730096


In [24]:
matchit_male_phecode.to_csv('matchit_phecode_matrix_male.csv', index = False)

### Load files

In [18]:
import pandas as pd
matchit_phecode = pd.read_csv('result_v2/matchit_phecode_matrix.csv')
# matchit_female_phecode = pd.read_csv('result_v2/matchit_phecode_matrix_female.csv')
# matchit_male_phecode = pd.read_csv('result_v2/matchit_phecode_matrix_male.csv')
# phecode_matrix_demo = pd.read_csv('result_v2/phecode_matrix_demo.csv')

In [24]:
a = matchit_phecode[matchit_phecode['Female']== 1].iloc[:,0:-16]

In [28]:
a.loc[:, a.sum(axis = 0) ==0]

,187,187.1,187.2,257,282,338,359,601.11,601.3,602.3,603.2,604.1,604.2,604.3,710.1,722,750.5,751.12
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24094,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
24095,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
24096,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
24099,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [29]:
b =  matchit_phecode[matchit_phecode['Male']== 1].iloc[:,0:-16]

In [12]:
matchit_female_phecode.iloc[:,0:-16]

,person_id,008,008.5,008.51,008.52,008.6,008.7,010,031,038,...,979,981,983,984,985,986,987,988,989,990
0,1000095,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1000151,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1000262,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1000357,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1000417,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18992,9988752,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18993,9992115,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
18994,9992814,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18995,9998092,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
b.loc[:, b.sum(axis = 0) ==0]

,1003,174,174.3,182,184.1,184.11,204.3,218.2,221,227,...,750,750.2,756.2,756.22,795.8,961.1,962.1,965.3,972.2,994
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24084,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24086,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24090,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24097,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
matchit_phecode.iloc[:,0:-16]

,person_id,008,008.5,008.51,008.52,008.6,008.7,010,031,038,...,984,985,986,987,988,989,990,994,994.1,994.2
0,1000095,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1000151,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1000174,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1000262,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1000344,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24096,9992814,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24097,9995309,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24098,9996889,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
24099,9998676,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
phecode_matrix_demo.iloc[:,0:-16]

,person_id,008,008.5,008.51,008.52,008.6,008.7,010,031,031.1,...,986,987,988,989,990,994,994.1,994.2,994.21,996
0,1000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1000004,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1000033,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
3,1000039,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1000042,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216937,9999351,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
216938,9999412,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
216939,9999476,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
216940,9999653,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Archive

In [41]:
# # Apply the mapping to the A column to create the B column
# matchit_cohort_df['sex_revised'] = matchit_cohort_df['sex_at_birth'].map(sex_mapping)
# matchit_cohort_df['race_revised'] = matchit_cohort_df['race'].map(race_mapping)
# matchit_cohort_df['ethinicity_revised'] = matchit_cohort_df['ethnicity'].map(ethinicity_mapping)

In [44]:
# # Create the binary matrix using pivot_table
# sex_matrix = matchit_cohort_df.assign(value=1).pivot_table(index='person_id', columns='sex_revised', values='value', fill_value=0)
# race_matrix = matchit_cohort_df.assign(value=1).pivot_table(index='person_id', columns='race_revised', values='value', fill_value=0)
# ethinicity_matrix = matchit_cohort_df.assign(value=1).pivot_table(index='person_id', columns='ethinicity_revised', values='value', fill_value=0)   

In [46]:
# # Apply z-score normalization
# matchit_cohort_df['age_normalized'] = stats.zscore(matchit_cohort_df['age_last_EHR'])

In [47]:
# is_hs = matchit_cohort_df[['person_id','HS']].drop_duplicates()

# matchit_demo_sub = (is_hs
#                    .merge(race_matrix, on='person_id', how='left')
#                    .merge(sex_matrix, on='person_id', how='left')
#                    .merge(ethinicity_matrix, on='person_id', how='left'))